# Final Fantasy Tactics Data to Neo4j

I've always wanted to learn more about graph databases, and what better way then to take a dataset and try to load it and extract the data. 

So when I get a chance to find data on Kaggle for a game that I love, in Final Fantasy Tactics for the Playstation, I knew I had to take the chance to load it into a graph database. It's a great way to learn how Cypher works.

[Source](https://www.kaggle.com/datasets/andrewparnell/final-fantasy-tactics-battles)

## Installing packages

Not just Pandas and Numpy is needed, you need the neo4j and graphdatascience package to access the database

In [1]:
# Install the rare packages needed
!pip install neo4j-driver
!pip install graphdatascience==1.0.0

In [2]:
# Importing the needed packages
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience
from myClassConnectToNeo4J import Neo4JConnection

In [3]:
# Connecting to the database (using localhost because I'm too cheap right now to get a Sandbox :P )
gds = Neo4JConnection()

In [4]:
# Clearing the data so I can start fresh
gds.clearDatabase()

True

## Loading the data into Dataframes

There are three csv files that need to be loaded
1. fft_jobs.csv - Contains all the jobs, the prerequisites and the stats added
1. fft_skills.csv - A collections of the jobs found, the skills per each, and how much job points(JP) are needed
1. fft_red-battles.csv - A collection of all the battles in the game, and the jobs of all the enemies

In [5]:
# Load all three tables into a dataframe to have a look
jobs       = pd.read_csv('data/fft_jobs.csv',sep=',',encoding = "ISO-8859-1")
redbattles = pd.read_csv('data/fft_red-battles.csv',sep=',',encoding = "ISO-8859-1")
skills     = pd.read_csv('data/fft_skills.csv',sep=',',encoding = "ISO-8859-1")

In [6]:
# For the level numbers, the next chapter is added as a new column so that it can be connected in order.
redbattles.insert(1, "nextchapter", 0)
redbattles['nextchapter'] = redbattles['chronological'].shift(-1)
redbattles['nextchapter'] = redbattles['nextchapter'].fillna(0)
redbattles['nextchapter'] = redbattles['nextchapter'].astype('int')

In [7]:
# Printing the shape, to confirm the size of the dataframes
print(f"The shape of the jobs table: {jobs.shape}")
print(f"The shape of the skills table: {skills.shape}")
print(f"The shape of the redbattles table: {redbattles.shape}")


The shape of the jobs table: (19, 24)
The shape of the skills table: (279, 4)
The shape of the redbattles table: (58, 23)


In [8]:
# Checking the stats of the jobs table 
display(jobs.describe())

,prereq_1_lvl,prereq_2_lvl,prereq_3_lvl,prereq_4_lvl,move,jump,speed,attack,magic,hp,mp,lvl_2_jp,lvl_3_jp,lvl_4_jp,lvl_5_jp,lvl_6_jp,lvl_7_jp,lvl_8_jp
count,17.000000,5.0,3.000000,1.0,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.0,19.0,19.0,19.0,19.0,19.0,19.0
mean,2.588235,4.0,2.333333,3.0,3.210526,3.210526,7.894737,2.789474,2.947368,2.578947,2.842105,200.0,350.0,550.0,800.0,1150.0,1550.0,2100.0
std,0.795206,0.0,0.577350,NaN,0.418854,0.418854,1.410072,0.917663,1.025978,0.837708,1.067872,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,2.000000,4.0,2.000000,3.0,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,1.000000,200.0,350.0,550.0,800.0,1150.0,1550.0,2100.0
25%,2.000000,4.0,2.000000,3.0,3.000000,3.000000,8.000000,2.000000,2.000000,2.000000,2.000000,200.0,350.0,550.0,800.0,1150.0,1550.0,2100.0
50%,2.000000,4.0,2.000000,3.0,3.000000,3.000000,8.000000,2.000000,3.000000,2.000000,3.000000,200.0,350.0,550.0,800.0,1150.0,1550.0,2100.0
75%,3.000000,4.0,2.500000,3.0,3.000000,3.000000,8.000000,4.000000,4.000000,3.000000,4.000000,200.0,350.0,550.0,800.0,1150.0,1550.0,2100.0
max,4.000000,4.0,3.000000,3.0,4.000000,4.000000,10.000000,4.000000,4.000000,4.000000,4.000000,200.0,350.0,550.0,800.0,1150.0,1550.0,2100.0


In [9]:
# Checking the stats of the jobs table 
display(skills.describe())

,jp
count,279.000000
mean,361.648746
std,274.236950
min,0.000000
25%,150.000000
50%,300.000000
75%,500.000000
max,1600.000000


In [10]:
# Checking the stats of the jobs table 
display(redbattles.describe())

,chronological,nextchapter,chapter,party_size,guests,enemy_1_level,enemy_2_level,enemy_3_level,enemy_4_level,enemy_5_level,enemy_6_level,enemy_7_level,enemy_8_level
count,58.000000,58.000000,58.000000,58.000000,58.000000,58.000000,53.000000,53.000000,50.000000,48.000000,44.000000,7.000000,2.000000
mean,29.500000,29.482759,2.896552,4.620690,0.775862,28.844828,27.245283,27.660377,27.380000,25.041667,25.659091,32.000000,25.000000
std,16.886879,16.916971,1.149976,0.812784,1.009483,18.178261,19.710311,20.303413,21.782684,17.205836,16.803852,14.236104,15.556349
min,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.000000,14.000000
25%,15.250000,15.250000,2.000000,4.000000,0.000000,13.000000,11.000000,12.000000,9.500000,10.000000,11.750000,21.000000,19.500000
50%,29.500000,29.500000,3.000000,5.000000,0.500000,27.500000,25.000000,26.000000,25.000000,23.000000,25.000000,33.000000,25.000000
75%,43.750000,43.750000,4.000000,5.000000,1.000000,40.250000,37.000000,37.000000,35.750000,36.250000,36.250000,41.000000,30.500000
max,58.000000,58.000000,4.000000,5.000000,5.000000,77.000000,89.000000,89.000000,94.000000,72.000000,72.000000,52.000000,36.000000


In [11]:
# Fixing all the Thief misspellings in the dataframe
for col in ['job', 'prereq_1_job', 'prereq_2_job','prereq_3_job', 'prereq_4_job']:
    jobs.loc[jobs[col] == 'Theif', col] = 'Thief'
for col in ['enemy_1_job', 'enemy_2_job','enemy_3_job', 'enemy_4_job','enemy_5_job', 'enemy_6_job','enemy_7_job', 'enemy_8_job']:
    redbattles.loc[redbattles[col] == 'Theif', col] = 'Thief'
for col in ['job']:
    skills.loc[skills[col] == 'Theif', col] = 'Thief'

## Loading all the data into the Neo4J

For this, it's time to load the data into the Neo4J Database. In this case, I could load it from Python, but I really wanted to practice and learn Cypher. I learned it in the best way possible, lots of trial, even more error. :D

### Creating a contraint

Here, this is where I can set the restraints of the nodes. This is what prevents duplicate records or other restrictions.

In [12]:
# The Constraint to confirm that the job titles are unique and not null
gds.runCypher("CREATE CONSTRAINT jobnameunique FOR (n:Job) REQUIRE n.title IS UNIQUE")
gds.runCypher("CREATE CONSTRAINT jobnamenull FOR (n:Job) REQUIRE n.title IS NOT NULL")

""


In [13]:
# The Constraint to confirm that the job titles are unique and not null
gds.runCypher("CREATE CONSTRAINT chapternameunique FOR (n:Chapter) REQUIRE n.title IS UNIQUE")
gds.runCypher("CREATE CONSTRAINT chapternamenull FOR (n:Chapter) REQUIRE n.title IS NOT NULL")

""


In [14]:
# The Constraint to confirm that the job titles are not null. There are skills with the same name but different type
# gds.runCypher("CREATE CONSTRAINT skillnameunique FOR (n:Skill) REQUIRE n.title IS UNIQUE")
gds.runCypher("CREATE CONSTRAINT skillnamenull FOR (n:Skill) REQUIRE n.title IS NOT NULL")

""


In [15]:
# The Constraint to confirm that the job titles are unique and not null
gds.runCypher("CREATE CONSTRAINT stagesnameunique FOR (n:Stages) REQUIRE n.chronological IS UNIQUE")
gds.runCypher("CREATE CONSTRAINT stagesnamenull FOR (n:Stages) REQUIRE n.chronological IS NOT NULL")

""


In [16]:
# The Constraint to confirm that the job titles are unique and not null
gds.runCypher("CREATE CONSTRAINT typenameunique FOR (n:Type) REQUIRE (n.title, n.type) IS UNIQUE")
gds.runCypher("CREATE CONSTRAINT typenamenull FOR (n:Type) REQUIRE n.title IS NOT NULL")

""


In [17]:
# The Constraint to confirm that the job titles are unique and not null
gds.runCypher("CREATE CONSTRAINT typelocationunique FOR (n:Locations) REQUIRE n.title IS UNIQUE")
gds.runCypher("CREATE CONSTRAINT typelocationnull FOR (n:Locations) REQUIRE n.title IS NOT NULL")

""


### Loading in the data

#### Loading the jobs database
This table has all the jobs and the prerequisites

In [18]:
# Load Jobs Nodes
gds.runCypher(
    """
LOAD CSV WITH HEADERS FROM 'file:///fft_jobs.csv' AS row
CREATE (a:Job{
    title:row["job"],
    move:toInteger( row["move"]),
    jump:toInteger( row["jump"]),
    speed:toInteger( row["speed"]),
    physical_evade:row["physical_evade"],
    attack:toInteger( row["attack"]),
    magic:toInteger( row["magic"]),
    hp:toInteger( row["hp"]),
    mp:toInteger( row["mp"])
})
RETURN a.title, a.move, a.jump, a.speed, a.physical_evade, a.attack, a.magic, a.hp, a.mp;
    """
)

,a.title,a.move,a.jump,a.speed,a.physical_evade,a.attack,a.magic,a.hp,a.mp
0,Squire,4,3,6,5%,2,1,3,1
1,Chemist,3,3,8,5%,2,3,2,3
2,Knight,3,3,8,10%,4,3,4,3
3,Archer,3,3,8,10%,3,2,3,2
4,Priest,3,3,9,5%,3,4,2,4
5,Wizard,3,3,8,5%,2,4,2,4
6,Monk,3,4,9,20%,4,3,4,2
7,Thief,4,4,9,25%,3,2,3,2
8,Oracle,3,3,8,5%,2,4,2,4
9,Time Mage,3,3,8,5%,2,4,2,4


There is a typo in the Samurai tab. It needs to be corrected before the skills connections.

In [19]:
# There is a typo in Samurai table, fixing for future matching
gds.runCypher(
    """
    MATCH(sp:Job{title:"Samuari"}) SET sp.title = "Samurai"
    return sp.title;
    """
)
#! End of Jobs File

,sp.title
0,Samurai


In [20]:
# Load Relations of jobs and prerequisites
for i in range(1,5):
    gds.runCypher(f"""
LOAD CSV WITH HEADERS FROM 'file:///fft_jobs.csv' AS row
WITH row WHERE row['prereq_{i}_job'] IS NOT NULL
    MATCH (x:Job {{title:row['job']}})
    MATCH (a:Job {{title:row['prereq_{i}_job']}})
    CREATE (a)-[:prerequisite_of {{min_level:toInteger( row['prereq_{i}_lvl'])}}]->(x)
RETURN a.title, x.title;
    """)

In [21]:
gds.runCypher("MATCH (a:Job)-[x:prerequisite_of]->(b:Job) RETURN a.title, type(x), x.min_level, b.title")

,a.title,type(x),x.min_level,b.title
0,Archer,prerequisite_of,2,Thief
1,Archer,prerequisite_of,3,Ninja
2,Chemist,prerequisite_of,2,Wizard
3,Chemist,prerequisite_of,2,Priest
4,Geomancer,prerequisite_of,2,Ninja
5,Geomancer,prerequisite_of,4,Dancer
6,Knight,prerequisite_of,2,Monk
7,Lancer,prerequisite_of,4,Dancer
8,Mediator,prerequisite_of,4,Bard
9,Monk,prerequisite_of,3,Geomancer


#### Loading the skills database

This table has all the skills connected to each job. This contains the category of each skill and the number of job points (JP) needed to buy the skill.

In [22]:
# Loads all Skill types
gds.runCypher(
    """
LOAD CSV WITH HEADERS FROM 'file:///fft_skills.csv' AS row
MERGE (t:Type{
    title:row["type"]
})
RETURN t.title;
    """
).head()

,t.title
0,Command
1,Command
2,Command
3,Command
4,Reaction


In [23]:
# This loads all skills Nodes
gds.runCypher(
    """
LOAD CSV WITH HEADERS FROM 'file:///fft_skills.csv' AS row
MERGE (s:Skill{
    title:row["name"],
    type: row["type"]
})
RETURN s.title;
    """
).head()

,s.title
0,Accumulate
1,Dash
2,Throw Stone
3,Heal
4,Counter Tackle


In [24]:
# This loads all relationships of jobs, skills and types
gds.runCypher(
    """
LOAD CSV WITH HEADERS FROM 'file:///fft_skills.csv' AS row
WITH row
    MATCH (j:Job {title:row['job']})
    MATCH (s:Skill {title:row['name'], type:row['type']})
    CREATE (j)-[x:has_skill {jp:toInteger(row["jp"])}]->(s)
RETURN j.title, type(x), x.jp, s.title;
    """
)
#! End of Skills file

,j.title,type(x),x.jp,s.title
0,Squire,has_skill,300,Accumulate
1,Squire,has_skill,80,Dash
2,Squire,has_skill,90,Throw Stone
3,Squire,has_skill,150,Heal
4,Squire,has_skill,180,Counter Tackle
...,...,...,...,...
274,Bard,has_skill,100,Last Song
275,Bard,has_skill,450,MA Save
276,Bard,has_skill,500,Face UP
277,Bard,has_skill,1000,Move +3


In [25]:
# This loads all relationships of jobs, skills and types
gds.runCypher(
    """
LOAD CSV WITH HEADERS FROM 'file:///fft_skills.csv' AS row
WITH row
    MATCH (t:Type {title:row['type']})
    MATCH (s:Skill {title:row['name'], type:row['type']})
    CREATE (s)-[y:is_type]->(t)
RETURN t.title, type(y), s.title;
    """
)
#! End of Skills file

,t.title,type(y),s.title
0,Command,is_type,Accumulate
1,Command,is_type,Dash
2,Command,is_type,Throw Stone
3,Command,is_type,Heal
4,Reaction,is_type,Counter Tackle
...,...,...,...
274,Command,is_type,Last Song
275,Reaction,is_type,MA Save
276,Reaction,is_type,Face UP
277,Movement,is_type,Move +3


In [26]:
# This loads all relationships of jobs, skills and types
gds.runCypher(
    """
    MERGE (j:Job)-[x:has_skill]->(s:Skill)
    RETURN j.title, type(x),x.jp, s.title
    """
)

,j.title,type(x),x.jp,s.title
0,Squire,has_skill,200,Move +1
1,Squire,has_skill,80,Dash
2,Squire,has_skill,90,Throw Stone
3,Squire,has_skill,50,Defend
4,Squire,has_skill,180,Counter Tackle
...,...,...,...,...
274,Bard,has_skill,1000,Move +3
275,Bard,has_skill,100,Last Song
276,Bard,has_skill,100,Angel Song
277,Bard,has_skill,100,Cheer Song


In [27]:
# This loads all relationships of jobs, skills and types
gds.runCypher(
    """
    MATCH (s:Skill)-[y:is_type]->(t:Type)
    RETURN s.title, type(y), t.title
    """
)

,s.title,type(y),t.title
0,Sword,is_type,Command
1,Wiznaibus,is_type,Command
2,Cure 2,is_type,Command
3,Throw Stone,is_type,Command
4,Katana,is_type,Command
...,...,...,...
274,Teleport,is_type,Movement
275,Move +3,is_type,Movement
276,Any Ground,is_type,Movement
277,Ignore Height,is_type,Movement


#### Loading the red_battle database

This table has all the chapters and battles, with all the jobs of all the enemies connected to it. 

In [28]:
# Loads all chapters in the game
gds.runCypher(
    """
    LOAD CSV WITH HEADERS FROM 'file:///fft_red-battles.csv' AS row
    MERGE (c:Chapter{
        title:toInteger(row["chapter"])
    })
    RETURN c.title;
    """
).head()

,c.title
0,1
1,1
2,1
3,1
4,1


In [29]:
# Loads all Stages in the game, and parameters
gds.runCypher(
    """
    LOAD CSV WITH HEADERS FROM 'file:///fft_red-battles.csv' AS row
    MERGE (l:Locations{
        title:row["location"]
    })
    RETURN l.title
    """
)

,l.title
0,Orbonne Monastery
1,Gariland Magic City
2,Mandalia Plains
3,Sweegy Woods
4,Slums of Dorter
5,Cellar of the Sand Rat
6,Thieves Fort
7,Lenalia Plateau
8,Windmill Shed
9,Fort Zeakden


In [30]:
# Loads all Stages in the game, and parameters
gds.runCypher(
    """
    LOAD CSV WITH HEADERS FROM 'file:///fft_red-battles.csv' AS row
    CREATE (s:Stages{
        chronological:row["chronological"],
        title:row["location"],
        win_condition:row["win_condition"],
        party_size:toInteger(row["party_size"]),
        guests:toInteger( row["guests"])
    })
    RETURN s.chronological, s.title, s.win_condition, s.party_size, s.guests
    """
)

,s.chronological,s.title,s.win_condition,s.party_size,s.guests
0,1,Orbonne Monastery,Defeat all enemies,1,5
1,2,Gariland Magic City,Defeat all enemies,5,1
2,3,Mandalia Plains,Defeat all enemies | Save ally,4,2
3,4,Sweegy Woods,Defeat all enemies,4,2
4,5,Slums of Dorter,Defeat all enemies,4,2
5,6,Cellar of the Sand Rat,Defeat all enemies,4,2
6,7,Thieves Fort,Defeat target,4,2
7,8,Lenalia Plateau,Defeat target,5,1
8,9,Windmill Shed,Defeat target,4,1
9,10,Fort Zeakden,Defeat target,4,1


In [31]:
# This loads all relationships of jobs, skills and types
gds.runCypher(
    """
    LOAD CSV WITH HEADERS FROM 'file:///fft_red-battles.csv' AS row
    WITH row
        MATCH (l:Locations {title:row['location']})
        MATCH (s:Stages {chronological:row["chronological"], title:row['location']})
        CREATE (s)-[x:located_at]->(l)
    RETURN l.title, type(x), s.title;
    """
)
#! End of Skills file

,l.title,type(x),s.title
0,Orbonne Monastery,located_at,Orbonne Monastery
1,Gariland Magic City,located_at,Gariland Magic City
2,Mandalia Plains,located_at,Mandalia Plains
3,Sweegy Woods,located_at,Sweegy Woods
4,Slums of Dorter,located_at,Slums of Dorter
5,Cellar of the Sand Rat,located_at,Cellar of the Sand Rat
6,Thieves Fort,located_at,Thieves Fort
7,Lenalia Plateau,located_at,Lenalia Plateau
8,Windmill Shed,located_at,Windmill Shed
9,Fort Zeakden,located_at,Fort Zeakden


In [32]:
# Loads all the relationships that put the stages in chronological order
for index, row in redbattles.iterrows():
    cyphertext = f"""
    MATCH (c:Stages {{ chronological: "{row['chronological']}" }})
    MATCH (s:Stages {{ chronological: "{row['nextchapter']}" }})
    CREATE (c)-[:next_battle]->(s)
    RETURN c.title,s.title;
    """
#     print(cyphertext)
    gds.runCypher(cyphertext)

In [33]:
# Testing the new next_battle relationship
gds.runCypher(
    """
    MATCH (c:Stages)-[x:next_battle]->(s:Stages)
    RETURN c.chronological, c.title, type(x), s.chronological, s.title;
    """
)

,c.chronological,c.title,type(x),s.chronological,s.title
0,1,Orbonne Monastery,next_battle,2,Gariland Magic City
1,10,Fort Zeakden,next_battle,11,Dorter Trade City
2,11,Dorter Trade City,next_battle,12,Araguay Woods
3,12,Araguay Woods,next_battle,13,Zirekile Falls
4,13,Zirekile Falls,next_battle,14,Zaland Fort City
5,14,Zaland Fort City,next_battle,15,Bariaus Hill
6,15,Bariaus Hill,next_battle,16,Zigolis Swamp
7,16,Zigolis Swamp,next_battle,17,Goud Machine City Slums
8,17,Goud Machine City Slums,next_battle,18,Bariaus Valley
9,18,Bariaus Valley,next_battle,19,Golgorand Execution Site


In [34]:
# Loads all connections to Chapters and Stages
gds.runCypher(
    """
    LOAD CSV WITH HEADERS FROM 'file:///fft_red-battles.csv' AS row
    WITH row
        MATCH (c:Chapter {title:toInteger(row["chapter"])})
        MATCH (s:Stages {chronological:row["chronological"]})
        CREATE (s)-[x:in_chapter]->(c)
    RETURN c.title, type(x), s.chronological, s.title;
    """
)

,c.title,type(x),s.chronological,s.title
0,1,in_chapter,1,Orbonne Monastery
1,1,in_chapter,2,Gariland Magic City
2,1,in_chapter,3,Mandalia Plains
3,1,in_chapter,4,Sweegy Woods
4,1,in_chapter,5,Slums of Dorter
5,1,in_chapter,6,Cellar of the Sand Rat
6,1,in_chapter,7,Thieves Fort
7,1,in_chapter,8,Lenalia Plateau
8,1,in_chapter,9,Windmill Shed
9,1,in_chapter,10,Fort Zeakden


In [35]:
# Loads all the jobs from the enemy pool, but only unique names, and making non-playable flag for jobs not in user pool
for i in range(1,9):
    cyphertext = f"""
    LOAD CSV WITH HEADERS FROM 'file:///fft_red-battles.csv' AS row
    WITH row WHERE row['enemy_{i}_job'] IS NOT NULL
        MERGE (a:Job{{title:row['enemy_{i}_job']}})
            ON CREATE SET a.NonPlayable = True
    RETURN count(a);
    """
    gds.runCypher(cyphertext)

In [36]:
# Hydra is spelled wrong, creating new node to fix relations below
gds.runCypher(
    """
        CREATE (a:Job{title:"Hydra", NonPlayable: True})
        return a.title, a.NonPlayable;
    """
)

,a.title,a.NonPlayable
0,Hydra,True


In [37]:
# Testing the Jobs table with the new NonPlayable stat
gds.runCypher(
    """
MATCH (c:Job)
RETURN c.title, c.move, c.jump, c.speed, c.physical_evade, c.attack, c.magic, c.hp, c.mp, c.NonPlayable
ORDER BY c.title;
    """
)

,c.title,c.move,c.jump,c.speed,c.physical_evade,c.attack,c.magic,c.hp,c.mp,c.NonPlayable
0,Apanda,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True
1,Archaeic Demon,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True
2,Archaic Demon,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True
3,Archer,3.0,3.0,8.0,10%,3.0,2.0,3.0,2.0,None
4,Assassin,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True
5,Bard,3.0,3.0,8.0,5%,2.0,4.0,2.0,3.0,None
6,Black Chocobo,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True
7,Black Goblin,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True
8,Black Mage,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True
9,Bomb,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,True


In [38]:
# Connects all the enemies into the jobs
for i in range(1,9):
    cyphertext = f"""
    LOAD CSV WITH HEADERS FROM 'file:///fft_red-battles.csv' AS row
    WITH row WHERE row['enemy_{i}_job'] IS NOT NULL
        MATCH (j:Job {{title:row['enemy_{i}_job']}})
        MATCH (s:Stages {{chronological:row["chronological"]}})
        CREATE (s)-[:has_enemy {{level:toInteger(row['enemy_{i}_level'])}}]->(j)
    RETURN *;
    """
    gds.runCypher(cyphertext)

In [39]:
# Checking the has_enemy relationship
gds.runCypher(
    """
MATCH (s:Stages)-[x:has_enemy]->(j:Job)
RETURN s.chronological, s.title, type(x), j.title
ORDER BY s.chronological;
    """
)

,s.chronological,s.title,type(x),j.title
0,1,Orbonne Monastery,has_enemy,Archer
1,1,Orbonne Monastery,has_enemy,Archer
2,1,Orbonne Monastery,has_enemy,Archer
3,1,Orbonne Monastery,has_enemy,Chemist
4,1,Orbonne Monastery,has_enemy,Knight
...,...,...,...,...
310,9,Windmill Shed,has_enemy,Knight
311,9,Windmill Shed,has_enemy,Monk
312,9,Windmill Shed,has_enemy,Monk
313,9,Windmill Shed,has_enemy,Chocobo


In [40]:
# There is a case where Hydra is spelled wrong, so this will fix it.
gds.runCypher(
    """
MATCH (n1:Stages)-[r1:has_enemy]->(jold:Job) WHERE n1.chronological = "55" AND (jold.title = "Hyudra" OR jold.title="Hyrda")
MATCH (jnew:Job) WHERE jnew.title = "Hydra"
CREATE (n1)-[r2:has_enemy]->(jnew)
SET r2=r1
DELETE r1, jold;
    """
)

""


In [41]:
# There is a case where Thief is spelled wrong, so this will fix it.
gds.runCypher(
    """
MATCH (n1:Stages)-[r1:has_enemy]->(jold:Job) WHERE n1.chronological = "42" AND jold.title = "Theif"
MATCH (jnew:Job) WHERE jnew.title = "Thief"
CREATE (n1)-[r2:has_enemy]->(jnew)
SET r2=r1
DELETE r1, jold;
    """
)
#! End of redbattle file

""
